# Protocol studies

## Setup scripts if we are under SWAN

In [ ]:
# Working in the right path
%cd /eos/project/d/da-and-diffusion-studies/DA_Studies/Simulations/Models/da_sixtrack

In [ ]:
# Install the libraries
import sys
!{sys.executable} -m pip install --user tqdm pynverse sixtrackwrap crank-nicolson-numba henon-map symplectic-map uncertainties
!export PYTHONPATH=$CERNBOX_HOME/.local/lib/python3.7/site-packages:$PYTHONPATH

## Library imports

In [1]:
%matplotlib widget

In [2]:
import numpy as np
from numba import njit
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import scipy.interpolate as interpolate
import scipy.integrate as integrate
from scipy.misc import derivative as derivative
from pynverse import inversefunc
import scipy.optimize as optimize
from uncertainties import ufloat
import itertools
import scipy

import ipywidgets as widgets
from IPython.display import display

# Personal libraries
import symplectic_map as sm
import henon_map as hm
import crank_nicolson_numba.nekhoroshev as nk
import crank_nicolson_numba.generic as cn

## Noise function setup

In [3]:
def standard_map_noise(n_samples, k, p0=1.0, th0=1.0, normed = True):
    """Make an array of correlated noise
    
    Parameters
    ----------
    n_samples : unsigned int
        number of elements
    k : float
        standard map k parameter
    p0 : float, optional
        p0 starting point, by default 1.0
    th0 : float, optional
        th0 starting point, by default 1.0
    
    Returns
    -------
    ndarray, ndarray
        th and p ndarrays of dimention (n_samples)
    """    
    p = np.empty(n_samples)
    th = np.empty(n_samples)
    p[0] = p0
    th[0] = th0
    for i in range(1, n_samples):
        p[i] = np.mod(p[i - 1] + k * np.sin(th[i - 1]), 2*np.pi)
        th[i] = np.mod(th[i - 1] + p[i], 2 * np.pi)
    noise = (p - np.pi) / np.pi
    if normed:
        noise -= np.average(noise)
        noise /= np.std(noise)
    return th, noise


@njit
def make_correlated_noise(n_elements, gamma=0.0):
    """Make an array of correlated noise
    
    Parameters
    ----------
    n_elements : unsigned int
        number of elements
    gamma : float, optional
        correlation coefficient, by default 0.0
    
    Returns
    -------
    ndarray
        the noise array
    """    
    noise = np.random.normal(0.0, 1.0, n_elements)
    if gamma != 0.0:
        for i in range(1, n_elements):
            noise[i] += gamma * noise[i - 1]
    return noise

### Noise visualization
Correlated noise

In [4]:
n1_fig, n1_axs = plt.subplots(2, 2)

n_elements = 100

g0 = 0.0
g1 = 0.25
g2 = 0.5
g3 = 0.75

n1_axs[0][0].plot(make_correlated_noise(n_elements, gamma=g0))
n1_axs[0][0].set_xlabel("$t$")
n1_axs[0][0].set_ylabel("$\\xi(t)$")
n1_axs[0][0].set_title("$\\gamma={}$".format(g0))

n1_axs[0][1].plot(make_correlated_noise(n_elements, gamma=g1))
n1_axs[0][1].set_xlabel("$t$")
n1_axs[0][1].set_ylabel("$\\xi(t)$")
n1_axs[0][1].set_title("$\\gamma={}$".format(g1))

n1_axs[1][0].plot(make_correlated_noise(n_elements, gamma=g2))
n1_axs[1][0].set_xlabel("$t$")
n1_axs[1][0].set_ylabel("$\\xi(t)$")
n1_axs[1][0].set_title("$\\gamma={}$".format(g2))

n1_axs[1][1].plot(make_correlated_noise(n_elements, gamma=g3))
n1_axs[1][1].set_xlabel("$t$")
n1_axs[1][1].set_ylabel("$\\xi(t)$")
n1_axs[1][1].set_title("$\\gamma={}$".format(g3))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Standard map

In [5]:
n2_fig, n2_axs = plt.subplots(2, 2)

n_elements = 5000

g0 = 1.5
g1 = 2.0
g2 = 5.0
g3 = 10.0

th, p = standard_map_noise(n_elements, g0)
n2_axs[0][0].scatter(th, p, s=1.0)
n2_axs[0][0].set_xlabel("$\\theta$")
n2_axs[0][0].set_ylabel("$p$")
n2_axs[0][0].set_title("$\\kappa={}$".format(g0))

th, p = standard_map_noise(n_elements, g1)
n2_axs[0][1].scatter(th, p, s=1.0)
n2_axs[0][1].set_xlabel("$\\theta$")
n2_axs[0][1].set_ylabel("$p$")
n2_axs[0][1].set_title("$\\kappa={}$".format(g1))

th, p = standard_map_noise(n_elements, g2)
n2_axs[1][0].scatter(th, p, s=1.0)
n2_axs[1][0].set_xlabel("$\\theta$")
n2_axs[1][0].set_ylabel("$p$")
n2_axs[1][0].set_title("$\\kappa={}$".format(g2))

th, p = standard_map_noise(n_elements, g3)
n2_axs[1][1].scatter(th, p, s=1.0)
n2_axs[1][1].set_xlabel("$\\theta$")
n2_axs[1][1].set_ylabel("$p$")
n2_axs[1][1].set_title("$\\kappa={}$".format(g3))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Baseline known results
### Setting up the map

In [6]:
omega_0 = 2.1
omega_1 = 1.3
omega_2 = 0.7

epsilon = 0.1
x_star = 4.0
delta = 0.0001
exponent = (2)  # 1 / (2)

n_iterations = 100000
barrier_radius = 3.0    # pure x

In [7]:
# Initial I distribution
mean = 3.0
sigma = 0.2

n_realizations = 10000

# When inspecting map characteristics
n_I_samplings = 1000
I_min = 0
I_max = cn.action(barrier_radius, 0)
skip_prints = 10

#### Crank-Nicolson engine

In [8]:
I_linspace = np.linspace(I_min, I_max, n_I_samplings)

engine = cn.cn_generic(I_min, I_max, cn.normed_normal_linspace(I_min, I_max, mean, sigma, n_I_samplings), 0.5, lambda I: cn.D_calculator(I, epsilon, x_star, delta, exponent))

data0 = engine.get_data()

iterating CN engine

In [9]:
engine.iterate(n_iterations)
data1 = engine.get_data()

Visualizing CN result

In [10]:
fig1, ax1 = plt.subplots()

ax1.plot(I_linspace, data0, label="t=0")
ax1.plot(I_linspace, data1, label="t={}".format(n_iterations))
ax1.legend()
ax1.set_xlabel("I")
ax1.set_ylabel("rho")
ax1.set_title("Crank-Nicolson evolution")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Crank-Nicolson evolution')

#### Setting up discrete map

In [11]:
x_0 = np.array([cn.I_norm_sampling_to_x(mean, sigma) for i in range(n_realizations)])
p_0 = np.zeros(n_realizations)

discrete_engine = sm.symplectic_map.generate_instance(omega_0, omega_1, omega_2, epsilon, x_star, delta, exponent, 0.0, barrier_radius, x_0, p_0)

#### Individual noise, $\gamma = 0.0$

In [31]:
discrete_engine.compute_personal_noise(n_iterations, gamma=0.0)

Comparison with Crank-Nicolson (perect matching is observed)

In [13]:
bins=60
data = discrete_engine.get_action()
filtered_data = data[data != 0.0]
weights = np.empty_like(filtered_data)
### USEFUL! ###
weights.fill((filtered_data.size/data.size) * bins / (filtered_data.max()-filtered_data.min()) / filtered_data.size)
###############

fig2, ax2 = plt.subplots()

ax2.hist(filtered_data, weights=weights, bins=bins, label="sm")
ax2.plot(I_linspace, engine.get_data(), label="cn")
ax2.legend()
ax2.set_xlabel("I")
ax2.set_ylabel("rho")
ax2.set_title("Symplectic map and expected crank nicolson integration\ndifferent noise realization for every particle $(\\gamma=0.0)$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Symplectic map and expected crank nicolson integration\ndifferent noise realization for every particle $(\\gamma=0.0)$')

#### Individual noise, $\gamma = 0.1$

In [14]:
discrete_engine.reset()

In [15]:
discrete_engine.compute_personal_noise(n_iterations, gamma=0.1)

comparison with CN data, small discreapancies are present!

In [16]:
bins=60
data = discrete_engine.get_action()
filtered_data = data[data != 0.0]
weights = np.empty_like(filtered_data)
### USEFUL! ###
weights.fill((filtered_data.size/data.size) * bins / (filtered_data.max()-filtered_data.min()) / filtered_data.size)
###############

fig3, ax3 = plt.subplots()

ax3.hist(filtered_data, weights=weights, bins=bins, label="sm")
ax3.plot(I_linspace, engine.get_data(), label="cn")
ax3.legend()
ax3.set_xlabel("I")
ax3.set_ylabel("rho")
ax3.set_title("Symplectic map and expected crank nicolson integration\ndifferent noise realization for every particle $(\\gamma=0.1)$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Symplectic map and expected crank nicolson integration\ndifferent noise realization for every particle $(\\gamma=0.1)$')

#### Individual noise, $\gamma=0.5$

In [17]:
discrete_engine.reset()

In [18]:
discrete_engine.compute_personal_noise(n_iterations, gamma=0.5)

Comparison with CN data, strong discrepancies are detected! Even though a "general behaviour" can be visualized...

In [19]:
bins=60
data = discrete_engine.get_action()
filtered_data = data[data != 0.0]
weights = np.empty_like(filtered_data)
### USEFUL! ###
weights.fill((filtered_data.size/data.size) * bins / (filtered_data.max()-filtered_data.min()) / filtered_data.size)
###############

fig4, ax4 = plt.subplots()

ax4.hist(filtered_data, weights=weights, bins=bins, label="sm")
ax4.plot(I_linspace, engine.get_data(), label="cn")
ax4.legend()
ax4.set_xlabel("I")
ax4.set_ylabel("rho")
ax4.set_title("Symplectic map and expected crank nicolson integration\ndifferent noise realization for every particle $(\\gamma=0.5)$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Symplectic map and expected crank nicolson integration\ndifferent noise realization for every particle $(\\gamma=0.5)$')

#### Common noise, $\gamma =0.0$

In [20]:
discrete_engine.reset()

In [21]:
noise = make_correlated_noise(n_iterations, gamma=0.0)
discrete_engine.compute_common_noise(noise)

CN comparison

In [22]:
bins=60
data = discrete_engine.get_action()
filtered_data = data[data != 0.0]
weights = np.empty_like(filtered_data)
### USEFUL! ###
weights.fill((filtered_data.size/data.size) * bins / (filtered_data.max()-filtered_data.min()) / filtered_data.size)
###############

fig5, ax5 = plt.subplots()

ax5.hist(filtered_data, weights=weights, bins=bins, label="sm")
ax5.plot(I_linspace, engine.get_data(), label="cn")
ax5.legend()
ax5.set_xlabel("I")
ax5.set_ylabel("rho")
ax5.set_title("Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(\\gamma=0.0)$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(\\gamma=0.0)$')

#### Common noise, $\gamma =0.1$

In [23]:
discrete_engine.reset()

In [24]:
noise = make_correlated_noise(n_iterations, gamma=0.1)
discrete_engine.compute_common_noise(noise)

CN comparison

In [25]:
bins=60
data = discrete_engine.get_action()
filtered_data = data[data != 0.0]
weights = np.empty_like(filtered_data)
### USEFUL! ###
weights.fill((filtered_data.size/data.size) * bins / (filtered_data.max()-filtered_data.min()) / filtered_data.size)
###############

fig6, ax6 = plt.subplots()

ax6.hist(filtered_data, weights=weights, bins=bins, label="sm")
ax6.plot(I_linspace, engine.get_data(), label="cn")
ax6.legend()
ax6.set_xlabel("I")
ax6.set_ylabel("rho")
ax6.set_title("Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(\\gamma=0.1)$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(\\gamma=0.1)$')

#### Common noise, $\gamma =0.5$

In [26]:
discrete_engine.reset()

In [27]:
noise = make_correlated_noise(n_iterations, gamma=0.5)
discrete_engine.compute_common_noise(noise)

CN comparison

In [28]:
bins=60
data = discrete_engine.get_action()
filtered_data = data[data != 0.0]
weights = np.empty_like(filtered_data)
### USEFUL! ###
weights.fill((filtered_data.size/data.size) * bins / (filtered_data.max()-filtered_data.min()) / filtered_data.size)
###############

fig7, ax7 = plt.subplots()

ax7.hist(filtered_data, weights=weights, bins=bins, label="sm")
ax7.plot(I_linspace, engine.get_data(), label="cn")
ax7.legend()
ax7.set_xlabel("I")
ax7.set_ylabel("rho")
ax7.set_title("Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(\\gamma=0.5)$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(\\gamma=0.5)$')

#### Common noise, extremely chaotic standard map

In [52]:
discrete_engine.reset()

In [53]:
noise = standard_map_noise(n_iterations, 50.0)[1]
discrete_engine.compute_common_noise(noise)

CN comparison

In [54]:
bins=60
data = discrete_engine.get_action()
filtered_data = data[data != 0.0]
weights = np.empty_like(filtered_data)
### USEFUL! ###
weights.fill((filtered_data.size/data.size) * bins / (filtered_data.max()-filtered_data.min()) / filtered_data.size)
###############

fig8, ax8 = plt.subplots()

ax8.hist(filtered_data, weights=weights, bins=bins, label="sm")
ax8.plot(I_linspace, engine.get_data(), label="cn")
ax8.legend()
ax8.set_xlabel("I")
ax8.set_ylabel("rho")
ax8.set_title("Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(standard map, \\kappa=50.0)$")

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(standard map, \\kappa=50.0)$')

#### Common noise, chaotic standard map

In [55]:
discrete_engine.reset()

In [56]:
noise = standard_map_noise(n_iterations, 10.0)[1]
discrete_engine.compute_common_noise(noise)

CN comparison

In [57]:
bins=60
data = discrete_engine.get_action()
filtered_data = data[data != 0.0]
weights = np.empty_like(filtered_data)
### USEFUL! ###
weights.fill((filtered_data.size/data.size) * bins / (filtered_data.max()-filtered_data.min()) / filtered_data.size)
###############

fig8, ax8 = plt.subplots()

ax8.hist(filtered_data, weights=weights, bins=bins, label="sm")
ax8.plot(I_linspace, engine.get_data(), label="cn")
ax8.legend()
ax8.set_xlabel("I")
ax8.set_ylabel("rho")
ax8.set_title("Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(standard map, \\kappa=10.0)$")

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(standard map, \\kappa=10.0)$')

#### Common noise, not-so-chaotic standard map

In [58]:
discrete_engine.reset()

In [59]:
noise = standard_map_noise(n_iterations, 2.0)[1]
discrete_engine.compute_common_noise(noise)

CN comparison

In [60]:
bins=60
data = discrete_engine.get_action()
filtered_data = data[data != 0.0]
weights = np.empty_like(filtered_data)
### USEFUL! ###
weights.fill((filtered_data.size/data.size) * bins / (filtered_data.max()-filtered_data.min()) / filtered_data.size)
###############

fig9, ax9 = plt.subplots()

ax9.hist(filtered_data, weights=weights, bins=bins, label="sm")
ax9.plot(I_linspace, engine.get_data(), label="cn")
ax9.legend()
ax9.set_xlabel("I")
ax9.set_ylabel("rho")
ax9.set_title("Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(standard map, \\kappa=2.0)$")

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(standard map, \\kappa=2.0)$')

#### Common noise, almost regular standard map

In [61]:
discrete_engine.reset()

In [62]:
noise = standard_map_noise(n_iterations, 1.5)[1]
discrete_engine.compute_common_noise(noise)

CN comparison

In [63]:
bins=60
data = discrete_engine.get_action()
filtered_data = data[data != 0.0]
weights = np.empty_like(filtered_data)
### USEFUL! ###
weights.fill((filtered_data.size/data.size) * bins / (filtered_data.max()-filtered_data.min()) / filtered_data.size)
###############

fig10, ax10 = plt.subplots()

ax10.hist(filtered_data, weights=weights, bins=bins, label="sm")
ax10.plot(I_linspace, engine.get_data(), label="cn")
ax10.legend()
ax10.set_xlabel("I")
ax10.set_ylabel("rho")
ax10.set_title("Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(standard map, \\kappa=1.5)$")

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Symplectic map and expected crank nicolson integration\nsame noise realization for every particle $(standard map, \\kappa=1.5)$')

## From all the above plots...what can we roughly say about our map with different noises?

* We can see how the increasing $\gamma$ factor (i.e. noise correlation), breaks the basic expected diffusive process
* We can see similar results by assigning to each particle the same noise realization (even though we observe some form of resonance here and there)
* With a (normalized) chaotic standard map, we observe the same behaviour
* With a not-so-chaotic standard map, we observe a complete breaking of the behaviour

**TODO::study the possibility of different time scales**

## Let's visualize the corresponding CN diffusion value

In [41]:
cn_fig, cn_ax = plt.subplots()

cn_ax.plot(engine.I, engine.diffusion)
cn_ax.set_xlabel("$I$")
cn_ax.set_ylabel("$D(I)$")
cn_ax.set_title("Corresponding Crank-Nicolson diffusion value for the map")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Corresponding Crank-Nicolson diffusion value for the map')

## Now, is there a valid tool for estimating the diffusion value on the discrete map process?

There is one, used by Mestre et al. in "Stochastic approach to diffusion inside the chaotic layer ofa resonance" (https://arxiv.org/pdf/1311.2635.pdf). It is based on a discrete evaluation of the displacement of a set of initial conditions with equal initial action value (eq. 15):

$$\mathcal{D}(I ; N) \equiv \frac{\left\langle\left(I_{N}-\left\langle I_{N}\right\rangle\right)^{2}\right\rangle}{N}$$

where $N$ is the number of iterations and $I_N$ the action value of the particle at the $N$-th iteration.

### Let's see how this equation behaves in our best-case scenario

In [17]:
samples = 50000
steps = 1

t2fig, t2ax = plt.subplots()

def diffusion_test_update(index):
    I0 = engine.I[index]
    start = np.sqrt(I0 * 2)

    angles = np.random.rand(samples) * np.pi * 2
    x0 = start * np.cos(angles)
    p0 = start * np.sin(angles)

    diff_test_plot = sm.symplectic_map.generate_instance(omega_0, omega_1, omega_2, epsilon, x_star, delta, exponent, 0.0, barrier_radius, x0, p0)
    
    time_list = []
    el_list = []

    j = 0
    for i in tqdm(range(250), leave=False):
        diff_test_plot.compute_personal_noise(steps, gamma=0.0)
        j += steps
        if np.count_nonzero(diff_test_plot.get_action()) < samples:
            print("Lost particles, stopping.")
            break
        time_list.append(j)
        el_list.append(np.var(diff_test_plot.get_action())/j)
    
    t2ax.clear()
    t2ax.plot(time_list, el_list, label="Discrete $D$ evaluation")
    t2ax.axhline(engine.diffusion[index], c="red", label="Actual $D(I)$ value in Crank-Nicolson")
    t2ax.legend()
    t2ax.set_xlabel("Iteration")
    t2ax.set_ylabel("Diffusion evaluation")
    t2ax.set_title("Discrete local diffusion evalutation for $I_0={:.3}$".format(I0))

widj_idx_I = widgets.IntSlider(value=500, min=0, max=len(engine.I), step=1, continuous_update=False)

ui = widgets.VBox([
    widgets.Label("I sample from CN engine"), widj_idx_I,
])
out = widgets.interactive_output(
    diffusion_test_update,
    {"index":widj_idx_I}
)

display(ui, out)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'engine' is not defined

#### A few comments on the plot(s) above
* Except for the extreme cases (extremely low $I$ values, under our numerical tolerance, and high $I$ values, for which we immediatly loose the particles), we always observe a good correspondance between the discrete $D$ evaluation and the theoretical expected $D$ value (less than $1\%$).
* We can observe how the error is random dependant (i.e. re-executing the same evaluation twice leads to very different results).
* However, the fluctuations are strong, and we can't push the itarations too much otherwise the diffusion we are measuring is no longer "local", but starts to be affected by the neighbour values.

### Can we use then this tool for making a proper estimation/reconstruction of $D(I)$?

Let's try to consider average value $\pm$ standard deviation:

In [43]:
samples = 50000
steps = 1
iters = 150

I_list = engine.I[300:-100:2]

averages = []
stds = []

for q, I0 in tqdm(list(enumerate(I_list))):
    start = np.sqrt(I0 * 2)

    angles = np.random.rand(samples) * np.pi * 2
    x0 = start * np.cos(angles)
    p0 = start * np.sin(angles)

    diff_test = sm.symplectic_map.generate_instance(omega_0, omega_1, omega_2, epsilon, x_star, delta, exponent, 0.0, barrier_radius, x0.copy(), p0.copy())
    
    time_list = []
    el_list = []
    
    j = 0
    for i in tqdm(range(iters)):
        diff_test.compute_personal_noise(steps, gamma=0.0)
        j += steps
        if (
            np.count_nonzero(diff_test.get_action()) < samples
           ):
            print("Lost particles, stopping.")
            break
        time_list.append(j)
        el_list.append(np.var(diff_test.get_action())/j)
        
    el_list = np.asarray(el_list)
    stds.append(np.std(el_list))
    averages.append(np.average(el_list)) 


In [44]:
t3fig, t3ax = plt.subplots()

t3ax.plot(I_list, engine.diffusion[300:-100:2], c="C1", label="$D$ theoretical value", linewidth=0.8)
t3ax.scatter(I_list, averages, s = 2, marker="x", color="C0", label="$D$ estimated")
t3ax.fill_between(I_list, np.asarray(averages) - np.asarray(stds), np.asarray(averages) + np.asarray(stds), color="C0", alpha=0.5, label="$D$ uncertainty")

t3ax.legend()
t3ax.set_xlabel("$I$")
t3ax.set_ylabel("$D(I)$")

t3ax.set_title("Comparison between theoretical $D(I)$ value\nand discrete reconstruction")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between theoretical $D(I)$ value\nand discrete reconstruction')

From what we can see... we have a pretty good reconstruction! (Excepct for a few critical values that probably generates some resonances with the chosen $\omega$ values...)

### Can this measure be used for visualizing the "performance" of the CN reconstruction?

In [45]:
samples = 10000
steps = 1
iters = 500

noise_common_0 = standard_map_noise(steps * iters, 1.5)[1]
noise_common_1 = standard_map_noise(steps * iters, 10.0)[1]
noise_common_2 = standard_map_noise(steps * iters, 100.0)[1]
noise_common_3 = make_correlated_noise(steps * iters, 0.0)

minimums = []
maximums = []
averages = []
stds = []

averages1 = []
stds1 = []
averages2 = []
stds2 = []
averages3 = []
stds3 = []
averages4 = []
stds4 = []

averages5 = []
stds5 = []
averages6 = []
stds6 = []
averages7 = []
stds7 = []
averages8 = []
stds8 = []

I_list = engine.I[-200:-100:10]

for q, I0 in tqdm(list(enumerate(I_list))):
    start = np.sqrt(I0 * 2)

    angles = np.random.rand(samples) * np.pi * 2
    x0 = start * np.cos(angles)
    p0 = start * np.sin(angles)

    diff_test = sm.symplectic_map.generate_instance(omega_0, omega_1, omega_2, epsilon, x_star, delta, exponent, 0.0, barrier_radius, x0.copy(), p0.copy())
    
    diff_test1 = sm.symplectic_map.generate_instance(omega_0, omega_1, omega_2, epsilon, x_star, delta, exponent, 0.0, barrier_radius, x0.copy(), p0.copy())
    diff_test2 = sm.symplectic_map.generate_instance(omega_0, omega_1, omega_2, epsilon, x_star, delta, exponent, 0.0, barrier_radius, x0.copy(), p0.copy())
    diff_test3 = sm.symplectic_map.generate_instance(omega_0, omega_1, omega_2, epsilon, x_star, delta, exponent, 0.0, barrier_radius, x0.copy(), p0.copy())
    diff_test4 = sm.symplectic_map.generate_instance(omega_0, omega_1, omega_2, epsilon, x_star, delta, exponent, 0.0, barrier_radius, x0.copy(), p0.copy())
    diff_test5 = sm.symplectic_map.generate_instance(omega_0, omega_1, omega_2, epsilon, x_star, delta, exponent, 0.0, barrier_radius, x0.copy(), p0.copy())
    diff_test6 = sm.symplectic_map.generate_instance(omega_0, omega_1, omega_2, epsilon, x_star, delta, exponent, 0.0, barrier_radius, x0.copy(), p0.copy())
    diff_test7 = sm.symplectic_map.generate_instance(omega_0, omega_1, omega_2, epsilon, x_star, delta, exponent, 0.0, barrier_radius, x0.copy(), p0.copy())
    diff_test8 = sm.symplectic_map.generate_instance(omega_0, omega_1, omega_2, epsilon, x_star, delta, exponent, 0.0, barrier_radius, x0.copy(), p0.copy())

    time_list = []
    el_list = []
    
    el_list1 = []
    el_list2 = []
    el_list3 = []
    el_list4 = []
    el_list5 = []
    el_list6 = []
    el_list7 = []
    el_list8 = []

    j = 0
    for i in tqdm(range(iters)):
        diff_test.compute_personal_noise(steps, gamma=0.0)
        diff_test1.compute_personal_noise(steps, gamma=0.1)
        diff_test2.compute_personal_noise(steps, gamma=0.2)
        diff_test3.compute_personal_noise(steps, gamma=0.3)
        diff_test4.compute_personal_noise(steps, gamma=0.5)
        diff_test5.compute_common_noise(noise_common_0[i * steps : (i+1) * steps])
        diff_test6.compute_common_noise(noise_common_1[i * steps : (i+1) * steps])
        diff_test7.compute_common_noise(noise_common_2[i * steps : (i+1) * steps])
        diff_test8.compute_common_noise(noise_common_3[i * steps : (i+1) * steps])
        j += steps
        if (
            np.count_nonzero(diff_test.get_action()) < samples or
            np.count_nonzero(diff_test1.get_action()) < samples or
            np.count_nonzero(diff_test2.get_action()) < samples or
            np.count_nonzero(diff_test3.get_action()) < samples or
            np.count_nonzero(diff_test4.get_action()) < samples or
            np.count_nonzero(diff_test5.get_action()) < samples or
            np.count_nonzero(diff_test6.get_action()) < samples or
            np.count_nonzero(diff_test7.get_action()) < samples or
            np.count_nonzero(diff_test8.get_action()) < samples
           ):
            print("Lost particles, stopping.")
            break
        time_list.append(j)
        el_list.append(np.var(diff_test.get_action())/j)
        el_list1.append(np.var(diff_test1.get_action())/j)
        el_list2.append(np.var(diff_test2.get_action())/j)
        el_list3.append(np.var(diff_test3.get_action())/j)
        el_list4.append(np.var(diff_test4.get_action())/j)
        el_list5.append(np.var(diff_test5.get_action())/j)
        el_list6.append(np.var(diff_test6.get_action())/j)
        el_list7.append(np.var(diff_test7.get_action())/j)
        el_list8.append(np.var(diff_test8.get_action())/j)
        
        
    el_list = np.asarray(el_list)
    minimums.append(np.min(el_list))
    maximums.append(np.max(el_list))    
    stds.append(np.std(el_list))
    averages.append(np.average(el_list)) 
    stds1.append(np.std(el_list1))
    averages1.append(np.average(el_list1)) 
    stds2.append(np.std(el_list2))
    averages2.append(np.average(el_list2)) 
    stds3.append(np.std(el_list3))
    averages3.append(np.average(el_list3)) 
    stds4.append(np.std(el_list4))
    averages4.append(np.average(el_list4)) 
    stds5.append(np.std(el_list5))
    averages5.append(np.average(el_list5)) 
    stds6.append(np.std(el_list6))
    averages6.append(np.average(el_list6))   
    stds7.append(np.std(el_list7))
    averages7.append(np.average(el_list7))   
    stds8.append(np.std(el_list8))
    averages8.append(np.average(el_list8))    

Lost particles, stopping.



Lost particles, stopping.




In [46]:
t4fig, t4ax = plt.subplots()

t4ax.plot(I_list, engine.diffusion[-200:-100:10], label="Theoretical $D$")

t4ax.errorbar(I_list, averages, yerr=stds, linewidth=0, elinewidth=1, capsize=1, label="Different realizations, $\\gamma = 0.0$")
t4ax.errorbar(I_list, averages1, yerr=stds1, linewidth=0, elinewidth=1, capsize=1, label="Different realizations, $\\gamma = 0.1$")
t4ax.errorbar(I_list, averages2, yerr=stds2, linewidth=0, elinewidth=1, capsize=1, label="Different realizations, $\\gamma = 0.2$")
t4ax.errorbar(I_list, averages3, yerr=stds3, linewidth=0, elinewidth=1, capsize=1, label="Different realizations, $\\gamma = 0.3$")
t4ax.errorbar(I_list, averages4, yerr=stds4, linewidth=0, elinewidth=1, capsize=1, label="Different realizations, $\\gamma = 0.5$")
t4ax.errorbar(I_list, averages5, yerr=stds5, linewidth=0, elinewidth=1, capsize=1, label="Standard map, low chaos $(\\kappa=1.5)$")
t4ax.errorbar(I_list, averages6, yerr=stds6, linewidth=0, elinewidth=1, capsize=1, label="Standard map, regular chaos $(\\kappa=10.0)$")
t4ax.errorbar(I_list, averages7, yerr=stds7, linewidth=0, elinewidth=1, capsize=1, label="Standard map, very high chaos $(\\kappa=100.0)$")
t4ax.errorbar(I_list, averages8, yerr=stds8, linewidth=0, elinewidth=1, capsize=1, label="Common realization, $\\gamma=0.0$")

t4ax.legend(fontsize="x-small")
t4ax.set_xlabel("$I$")
t4ax.set_ylabel("$D(I)$")
t4ax.set_title("Various $D(I)$ discrete measurements")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Various $D(I)$ discrete measurements')

# What about Hénon?
We want to probe our discrete diffusion measurement in a modulated Hénon map and see what we can obtain as diffusion values.

Can the resulting $D(I)$ values be used for a 1D FP process reconstruction?

In [12]:
import henon_map as hm

In [20]:
samples = 100
steps = 10000

I = 0.09
radius = np.sqrt(I * 2)
print(radius)
radiuses = np.ones(samples) * radius
henon_engine = hm.full_track.generate_instance(
    radiuses,
    np.random.rand(samples) * np.pi / 2,
    np.random.rand(samples) * np.pi * 2,
    np.random.rand(samples) * np.pi * 2,
    np.ones(samples, dtype=np.int) * steps,
    16.0
)

x, px, y, py = henon_engine.compute() # niters, nsamples
r, _, _, _ = hm.cartesian_to_polar(x, px, y, py)

final_I = (r ** 2 / 2)[:]
discrete_d = np.var(final_I, axis=1) #/ np.arange(1, final_I.shape[0] + 1)

0.4242640687119285


In [21]:
plt.figure()
plt.plot(discrete_d)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [119]:
plt.figure()
plt.hist(final_I[340],alpha=0.2, range=(0,0.06), bins=30)
plt.hist(final_I[341],alpha=0.2, range=(0,0.06), bins=30)
plt.hist(final_I[342],alpha=0.2, range=(0,0.06), bins=30)
plt.hist(final_I[-1],alpha=0.2, range=(0,0.06), bins=30)

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 7.700e+01,
        4.110e+02, 1.227e+03, 2.714e+03, 2.884e+03, 1.363e+03, 8.030e+02,
        2.940e+02, 1.420e+02, 6.800e+01, 1.600e+01, 1.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00]),
 array([0.   , 0.002, 0.004, 0.006, 0.008, 0.01 , 0.012, 0.014, 0.016,
        0.018, 0.02 , 0.022, 0.024, 0.026, 0.028, 0.03 , 0.032, 0.034,
        0.036, 0.038, 0.04 , 0.042, 0.044, 0.046, 0.048, 0.05 , 0.052,
        0.054, 0.056, 0.058, 0.06 ]),
 <a list of 30 Patch objects>)

In [138]:
I_max = 0.07
I_min = 0.01
I_steps = 100

I_list = np.linspace(I_min, I_max, I_steps)

samples = 5000
steps = 500

d_data = []
avg = []
std = []

for I in tqdm(I_list):
    radiuses = np.ones(samples) * (np.sqrt(I * 2))
    henon_engine = hm.full_track.generate_instance(
        radiuses,
        np.random.rand(samples) * np.pi / 2,
        np.random.rand(samples) * np.pi * 2,
        np.random.rand(samples) * np.pi * 2,
        np.ones(samples, dtype=np.int) * steps,
        16.0
    )
    
    x, px, y, py = henon_engine.compute() # niters, nsamples
    r, _, _, _ = hm.cartesian_to_polar(x, px, y, py)

    final_I = (r ** 2 / 2)[1:]
    discrete_d = np.var(final_I, axis=1) #/ np.arange(1, final_I.shape[0] + 1)
    d_data.append(discrete_d)
    avg.append(np.average(discrete_d))
    std.append(np.std(discrete_d))
    

In [139]:
plt.figure()
plt.plot(I_list, avg)
plt.fill_between(I_list, np.asarray(avg) - np.asarray(std), np.asarray(avg) + np.asarray(std), alpha=0.2)

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
plt.figure()
plt.plot(discrete_d[:])

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …